In [34]:
import torch
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [11]:
class ResidualNet(torch.nn.Module):
    # 需要保证输出层的大小不变，卷积核是3 ， 需要填充 padding 1  因为 7- 3 + 1 = 5
    def __init__(self,channel):
        super(ResidualNet,self).__init__()
        self.conv_1 = torch.nn.Conv2d(channel,channel,kernel_size=(3,3),padding=1)
        self.conv_2 = torch.nn.Conv2d(channel,channel,kernel_size=(3,3),padding=1)
        
    def forward(self,x):
        y = F.relu(self.conv_1(x))
        y = self.conv_2(y)
        return y + x

    
r = ResidualNet(2)
a = torch.Tensor([1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9]).view(1,2,3,3)
y = r(a)
y

tensor([[[[1.1678, 2.0073, 2.6140],
          [3.7471, 5.0567, 5.6851],
          [7.0861, 8.9127, 9.0825]],

         [[1.4829, 3.0897, 3.9202],
          [4.0649, 5.8924, 6.7603],
          [6.9386, 8.3186, 9.3487]]]], grad_fn=<AddBackward0>)

In [45]:
# im = Image.open("1.png")
# transform = transforms.ToTensor()
# tensor = transform(im)
# print(tensor.shape)
train_data = datasets.FashionMNIST("ai_data",train=True,download=False,transform=transforms.ToTensor())
test_data = datasets.FashionMNIST("ai_data",train=False,download=False,transform=transforms.ToTensor())
train_loader = DataLoader(train_data,batch_size=32,shuffle=True,num_workers=1)
test_loader = DataLoader(train_data,batch_size=32,shuffle=True,num_workers=1)

In [57]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Conv2d(1,16,kernel_size=(5,5)),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2),
            ResidualNet(16),
            
            torch.nn.Conv2d(16,32,kernel_size=(5,5)),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d((2,2)),
            ResidualNet(32),
        )
        # self.conv_1 = torch.nn.Conv2d(1,16,kernel_size=(5,5))
        # self.conv_2 = torch.nn.Conv2d(16,32,kernel_size=(5,5))
        # self.pool_max = torch.nn.MaxPool2d((2,2))
        # self.r_block_1 = ResidualNet(16)
        # self.r_block_2 = ResidualNet(32)
        
    def forward(self, x):
        x = self.model(x)
        # x = self.pool_max(F.relu(self.conv_1(x)))
        # x = self.r_block_1(x)
        # x = self.pool_max(F.relu(self.conv_2(x)))
        # x = self.r_block_2(x)
        return x
        

In [58]:
a = Net()
for i in train_loader:
    x , y = i
    y_hat = a(x)
    print(y_hat.shape)
    break




torch.Size([32, 32, 4, 4])
